# k-Nearest Neighbor (kNN) Playground

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

The kNN classifier consists of two stages:

- During training, the classifier takes the training data and simply remembers it
- During testing, kNN classifies every test image by comparing to all training images and transfering the labels of the k most similar training examples
- The value of k is cross-validated

In this exercise you will implement these steps and understand the basic Image Classification pipeline, cross-validation, and gain proficiency in writing efficient, vectorized code.

In [ ]:
# Run some setup code for this notebook.

import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Load the raw CIFAR-10 data.
#cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'#
cifar10_dir = '/home/antoine/Documents/01_Tensorflow_Tensorboard_Programs/CIFAR10_data'

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# Subsample the data for more efficient code execution in this exercise
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

In [ ]:
# Reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

In [ ]:
from cs231n.classifiers import KNearestNeighbor

# Create a kNN classifier instance. 
# Remember that training a kNN classifier is a noop: 
# the Classifier simply remembers the data and does no further processing 
classifier = KNearestNeighbor()

## I - Testing the k-NearestNeighbor on Simple Dataset

In [ ]:
# Train dataset:

# Features :
X1 = np.array([[1,2], [3,4], [2,2], [2,5], [1,9], [2,10], [3,1], [4,5], [3,8], [4,2]]).T
X2 = np.array([[7,9], [8,9], [7,1], [6,5], [5,7], [5,1], [8,4], [7,12], [7,5], [8,15]]).T
X3 = np.array([[4,20], [3,13], [5,12], [4,16], [6,15], [5,15], [5,19], [3,17], [2,15], [6,19]]).T

# Labels : 
y1 = np.zeros(10)
y2 = np.ones(10)
y3 = np.ones(10)*2

X = np.hstack([X1,X2,X3]).T 
y = np.hstack([y1,y2,y3])

print("labels:",y.shape)
print("features:",X.shape)

In [ ]:
# Training on the dataset:
classifier.train(X, y)

In [ ]:
from matplotlib.colors import ListedColormap
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])

k       = 3
h       = .05
xx, yy  = np.meshgrid(np.arange(0,9,h), np.arange(0,21,h))
zz      = np.c_[xx.ravel(), yy.ravel()]

dists   = classifier.compute_distances_no_loops(zz)
y_preds = classifier.predict_labels(dists, k=k)
y_preds = y_preds.reshape(xx.shape)

print(zz.shape)
print(dists.shape)
print(y_preds.shape)

plt.figure()
plt.pcolormesh(xx, yy, y_preds, cmap=cmap_light)
plt.scatter(X1[0],X1[1], c='r')
plt.scatter(X2[0],X2[1], c='g')
plt.scatter(X3[0],X3[1], c='b')
plt.title("{}-NearestNeighbor Algorithm".format(k))
plt.xlabel("x")
plt.ylabel("y")
plt.axis('equal')
plt.show()


## II - Testing the k-NearestNeighbor on CIFAR10
### II0 - Introduction

In [ ]:
N = 15
X = X_train[10:10+N]
Y = y_train[10:10+N]
print("X:", X.shape)
print("Y:", Y)

# Plot1 ========================== :
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(X[i].reshape(32,32,3).astype("uint8"))
    plt.title(Y[i])
    plt.axis("off")
plt.show()

# Plot2 ============================ :
# Learning how to sort the features array X in function of the labels array Y:
idxs = np.argsort(Y)
X = X[idxs]

for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(X[i].reshape(32,32,3).astype("uint8"))
    plt.title(np.sort(Y)[i])
    plt.axis("off")
plt.show()

### IIA - Playing with the training set

In [ ]:
k = 6
N = 100
X = X_train[:N]
Y = y_train[:N]

###############################
# if using the simple dataset #
# N = 30                      #
# Y = y                       #
###############################

sorted_Y = np.sort(Y)


nn = KNearestNeighbor()

# Creating the ideal result:
ideal = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        ideal[i,j] = 1 if sorted_Y[i]==sorted_Y[j] else 0
        
# Computing distances:
nn.train(X, Y)
D1 = nn.compute_distances_no_loops(X)

# Computing distances of ordered X values:
idxs = np.argsort(Y)
X = X[idxs]
nn.train(X, sorted_Y)
D2 = nn.compute_distances_no_loops(X)

# Predictions:
Y_pred = nn.predict_labels(D2, k=k)
res = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        res[i,j] = Y_pred[i]==sorted_Y[j]

        
# Plotting results:

# Plot1:
plt.imshow(-D1, interpolation='none', cmap='RdYlGn')
plt.xlabel("X_train")
plt.ylabel("X_train")
plt.title("Distances")
plt.show()

# plot2:
plt.imshow(-D2, interpolation='none', cmap='RdYlGn')
plt.xlabel("X_train")
plt.ylabel("X_train")
plt.title("Ordered Distances")
plt.show()

# Plot3:
extent = 0, N, N, 0
plt.figure()
plt.imshow(-D2, interpolation='none', cmap='RdYlGn', extent=extent)
plt.imshow(-ideal, interpolation='none', cmap='Greys', alpha=0.3, extent=extent)
plt.xlabel("X_train")
plt.ylabel("X_train")
plt.title("Ordered Distances with labels overlayed")
plt.show()

# plot4:
extent = 0, N, N, 0
plt.figure()
plt.imshow(res, interpolation='none', cmap='RdYlGn', extent=extent)
plt.imshow(-ideal, interpolation='none', cmap='Greys', alpha=0.2, extent=extent)
plt.xlabel("X_train")
plt.ylabel("X_train")
plt.title("{}-NearestNeighbor Predictions".format(k))
plt.show()

### IIB - Playing with the testing set

In [ ]:
sorted_y_train = np.sort(y_train)
sorted_y_test  = np.sort(y_test)

# Creating the ideal result:
ideal = np.zeros((num_test, num_training))
for i in range(num_test):
    for j in range(num_training):
        ideal[i,j] = 1 if sorted_y_test[i]==sorted_y_train[j] else 0
        
# plot:
plt.figure()
plt.imshow(ideal, interpolation='none', cmap='Greys', alpha=0.9)
plt.xlabel("X train")
plt.ylabel("X test")
plt.title("Ideal Results".format(k))
plt.show()

In [ ]:
classifier.train(X_train, y_train)
D1 = classifier.compute_distances_no_loops(X_test)

# plot:
plt.figure()
plt.imshow(D1[:100,:], interpolation='none', cmap='Greys', alpha=0.9)
plt.xlabel("X test")
plt.ylabel("X train")
plt.title("Distances".format(k))
plt.show()

In [ ]:
idxs_train = np.argsort(y_train)
idxs_test  = np.argsort(y_test)

D2 = D1[idxs_test,:]
D2 = D2[:,idxs_train]

# plot:
plt.figure()
plt.imshow(D2, interpolation='none', cmap='Greys', alpha=0.9)
plt.xlabel("X train")
plt.ylabel("X test")
plt.title("Distances".format(k))
plt.show()

In [ ]:
k       = 10
y_preds = classifier.predict_labels(D1, k=k)

# Predictions:
predictions = np.zeros((num_test, num_training))
for i in range(num_test):
    for j in range(num_training):
        predictions[i,j] = (y_preds[i]==y_train[j])

# plot1:
plt.figure()
plt.imshow(predictions, interpolation='none', cmap='Greys', alpha=0.9)
plt.xlabel("X train")
plt.ylabel("X test")
plt.title("{}-NearestNeighbor".format(k))
plt.show()        
         
# Ordering the array:
sorted_predictions = predictions[idxs_test,:]
sorted_predictions = predictions[:,idxs_train]

# plot2:
extent = 0, num_training, num_test, 0
plt.figure()
plt.imshow(sorted_predictions, interpolation='none', cmap='Greys', extent=extent)
plt.imshow(-ideal, interpolation='none', cmap='Greys', alpha=0.2, extent=extent)
plt.xlabel("X train")
plt.ylabel("X test")
plt.title("{}-NearestNeighbor".format(k))
plt.show()

In [ ]:
# Compute accuracy:
accuracy = np.sum(y_preds == y_test) / float(num_test) *100
print("Accuracy: {:.2f}%".format(accuracy))